# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1:

#### Use the Notebook to build the code to scrape the following Wikipedia page,
#### https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
#### in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src="po_table.png" width="50%" height="750%"></img>

#### To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

## import the necessary libraries

In [49]:
#!conda install -c conda-forge beautifulsoup4 --yes # get BeautifulSoup package

import pandas as pd # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import requests # library to handle requests

from bs4 import BeautifulSoup # to parse html file

print('Libraries imported.')

Libraries imported.


## get postcode data for toronto from wikipedia

In [2]:
# get data from wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# convert to beautiful soup 
soup = BeautifulSoup(source, 'lxml') ##'html.parser') #'lxml')
#print(soup)

In [3]:
# this method return the index when element is found
# and return -1 when element is not found
def find_in_list(lst, value):
    try:
        return lst.index(value)
    except:
        return -1

In [4]:
# find the table in the page
table = soup.find('table', class_="wikitable sortable")

# define lists to store values in each table column
pc=[] # list for postcode
bh=[] # list for borough
nd=[] # list for neighborhood

# loop through each row in the wiki table
for row in table.find_all("tr"):
    try:
        # get the column values in each row
        col = row.find_all("td")
        
        # rule #1: Ignore cells with a borough that is Not assigned.
        if col[1].text != "Not assigned" :
            # rule #2: when multiple neighborhood can exist in one postalcode area, 
            #          combine neignbordhood with commas
            idx = find_in_list(pc, col[0].text)
            #print (idx)
            if idx > 0:
                nd[idx] = nd[idx] + "," + col[2].text.replace('\n','') # clean column value
                
            else:
                pc.append(col[0].text)
                bh.append(col[1].text)
                # rule #3: If a cell has a borough but a 'Not assigned' neighborhood, 
                #          then the neighborhood will be the same as the borough.
                if col[2].text.replace('\n','') == "Not assigned":
                    nd.append(col[1].text)
                else:
                    nd.append(col[2].text.replace('\n','')) # clean column value
            
    except Exception as e:
        pass

# define column names    
column=['PostalCode', 'Borough', 'Neighborhood']

# create a dataframe from the data
tto=pd.DataFrame({column[0]:pc,column[1]:bh, column[2]:nd})

# display some rows in the dataframe
print(tto.head())

# save dataframe to csv file
tto.to_csv('temp_toronto_postcodes_borough_neighborhood.csv',index=None)


  PostalCode           Borough                     Neighborhood
0        M3A        North York                        Parkwoods
1        M4A        North York                 Victoria Village
2        M5A  Downtown Toronto         Harbourfront,Regent Park
3        M6A        North York  Lawrence Heights,Lawrence Manor
4        M7A      Queen's Park                     Queen's Park


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
# print the dataframe's size 
print(tto.shape)

(103, 3)


In [6]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
##from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
%pip install geocoder

Note: you may need to restart the kernel to use updated packages.


## Part 2 : Use the Geocoder package or the csv file to create the following dataframe:


<img src="Part2.png" width="50%" height="750%"></img>

#### Install geocoder

#### Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [7]:
import geocoder # import geocoder

In [30]:
# initialize your variable to None
lat_lng_coords = None

# define list to store the post code and its corresponding latitude and longitudes
lst=[]
for postal_code in pc:
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        sleep(1) 
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    lst.append(print("{},{},{}".format(postal_code,latitude,longitude)))

import csv
#write this information to a csv file
with open('Geospatial_Coordinates.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['Postal Code','Latitude','Longitude'])
    for row in lst:
        r = row.split(',')
        csv_writer.writerow([r[0],r[1],r[2]])


#### !! unable to extract data from geocoder, so using 'Geospatial_Coordinates.csv'

In [8]:
# coordinates dataframe
coordDf = pd.read_csv('Geospatial_Coordinates.csv')
coordDf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Rename the common column in coordinate data frame

In [9]:
coordDf.rename(columns={"Postal Code": "PostalCode"}, inplace=True) ## 
coordDf.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# post code, borough and neighborhood
tto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


#### Merge the two dataframes

In [11]:
mg = pd.merge(tto, coordDf, on='PostalCode' )
mg.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Total records

In [12]:
mg.shape

(103, 5)

## Part 3:

#### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

 Just make sure:

 1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
 2. to generate maps to visualize your neighborhoods and how they cluster together.
 Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. 

#### Let's download all the dependencies that we will need.

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Filter the merged data frame for boroughs that contain the word Toronto

In [83]:
fdf = mg[mg['Borough'].str.contains("Toronto")].reset_index(drop=True)
fdf.head()
#fdf.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Identify unique boroughs and neighborhood counts

In [24]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(fdf['Borough'].unique()),
        fdf.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [25]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [109]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(fdf['Latitude'], fdf['Longitude'], fdf['Borough'], fdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
#map_toronto

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [26]:
CLIENT_ID = 'XGMCX5BO5EASJ1YG2ZYTMQJKLJJCK0HKLA1TNGVPYBWUHP2R' # your Foursquare ID
CLIENT_SECRET = 'QCMB52KBXKPAQUAOPJP44M4H1LQUSPMAZVJCFRHYF1JHH4WL'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XGMCX5BO5EASJ1YG2ZYTMQJKLJJCK0HKLA1TNGVPYBWUHP2R
CLIENT_SECRET:QCMB52KBXKPAQUAOPJP44M4H1LQUSPMAZVJCFRHYF1JHH4WL


#### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [27]:
fdf.loc[0, 'Neighborhood']

'Harbourfront,Regent Park'

#### Get the neighborhood's latitude and longitude values.

In [28]:
neighborhood_latitude = fdf.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = fdf.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = fdf.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in 'Harbourfront,Regent Park' within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [29]:
radius=500
LIMIT=100

#Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XGMCX5BO5EASJ1YG2ZYTMQJKLJJCK0HKLA1TNGVPYBWUHP2R&client_secret=QCMB52KBXKPAQUAOPJP44M4H1LQUSPMAZVJCFRHYF1JHH4WL&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=100'

##### Send the GET request and examine the resutls

In [32]:
results = requests.get(url).json()
#results

#### From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### Now we are ready to clean the json and structure it into a pandas dataframe.

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


##### And how many venues were returned by Foursquare?

In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


#### Explore Neighborhoods in Manhattan
##### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [37]:
# type your answer here
toronto_venues = getNearbyVenues(names=fdf['Neighborhood'],
                                   latitudes=fdf['Latitude'],
                                   longitudes=fdf['Longitude']
                                  )


Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


#### Let's check the size of the resulting dataframe

In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(1707, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Let's check how many venues were returned for each neighborhood

In [108]:
#toronto_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


#### Analyze Each Neighborhood

In [84]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

#### And let's examine the new dataframe size.

In [43]:
toronto_onehot.shape

(1707, 238)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [91]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

#### Let's confirm the new size

In [45]:
toronto_grouped.shape

(38, 238)

#### Let's print each neighborhood along with the top 5 most common venues

In [86]:
num_top_venues = 4

for hood in toronto_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

#### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [74]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

#### Cluster Neighborhoods
Run k-means to cluster the neighborhood into clusters.

In [102]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

##### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [103]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = fdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.head() # check the last columns!

##### Finally, let's visualize the resulting clusters

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

##### Cluster 1

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,-79.293031,0,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
28,Central Toronto,-79.383160,0,Restaurant,Playground,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


##### Cluster 2

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,-79.388790,1,Bus Line,Park,Swim School,Wings Joint,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
20,Central Toronto,-79.411307,1,Park,Trail,Sushi Restaurant,Home Service,Jewelry Store,Dumpling Restaurant,Eastern European Restaurant,Donut Shop,Dessert Shop,Electronics Store
32,Downtown Toronto,-79.377529,1,Park,Playground,Trail,Building,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


##### Cluster 3

In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.360636,2,Coffee Shop,Bakery,Park,Mexican Restaurant,Gym / Fitness Center,Breakfast Spot,Restaurant,Café,Pub,Theater
1,Downtown Toronto,-79.378937,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Restaurant,Italian Restaurant,Ramen Restaurant,Diner,Bookstore
2,Downtown Toronto,-79.375418,2,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Clothing Store,Gastropub,Cosmetics Shop,Beer Bar,Breakfast Spot
4,Downtown Toronto,-79.373306,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Bakery,Steakhouse,Farmers Market,Cheese Shop,Café,Clothing Store
5,Downtown Toronto,-79.387383,2,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Sandwich Place,Burger Joint,Indian Restaurant,Chinese Restaurant,Salad Place,Bar
6,Downtown Toronto,-79.422564,2,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Coffee Shop
7,Downtown Toronto,-79.384568,2,Coffee Shop,Café,American Restaurant,Bar,Steakhouse,Thai Restaurant,Hotel,Restaurant,Burger Joint,Bakery
8,West Toronto,-79.442259,2,Pharmacy,Bakery,Supermarket,Middle Eastern Restaurant,Portuguese Restaurant,Brewery,Fast Food Restaurant,Bus Stop,Discount Store,Café
9,Downtown Toronto,-79.381752,2,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Brewery,Bakery,Scenic Lookout,Pizza Place,Sporting Goods Shop
10,West Toronto,-79.419750,2,Bar,Coffee Shop,Asian Restaurant,Bakery,French Restaurant,Cocktail Bar,Pizza Place,Men's Store,Café,Boutique


##### Cluster 4

In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,-79.416936,3,Pool,Garden,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## the end.